In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
import json
import spacy
from spacy.matcher import PhraseMatcher
from ipynb.fs.full.helper import get_text_from_csv,clean_text,check_null,read_fb,check_null,find_words,find_topic,get_word_lable,count_topic,count_word


# GOOGLE + FACEBOOK

In [2]:
google_df = get_text_from_csv('GoogleAds/NY.csv')
google_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117 entries, 0 to 116
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   text             117 non-null    object
 1   advertiser_name  117 non-null    object
 2   platform         117 non-null    object
dtypes: object(3)
memory usage: 2.9+ KB


In [3]:
google_df.head()

,text,advertiser_name,platform
0,"""Representative for New York's First Congressi...",REP. LEE ZELDIN - U.S. HOUSE OF REPRESENTATIVES,google
1,"""New Yorkers deserve a governor who works as h...",Friends for Kathy Hochul,google
2,"""\""Help Khaled \"" to fight the three enemies o...",KHALED SALEM,google
3,"""\""Help Khaled \"" to fight the three enemies o...",KHALED SALEM,google
4,"""“Simply Put, your vote count to get Free educ...",KHALED SALEM,google


In [25]:
facebook_df = read_fb('FacebookAds/New York.csv')
facebook_df

text  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [28]:
facebook_df = clean_text(facebook_df)
facebook_df.head(20)

,text,advertiser_name,platform
0,over the past two years we have witnessed how covid 19 deepened the financial need of organizations in our community we depend on while highlighting gaps that existed before the public health emergency in march president biden signed a government funding law that included 21 9 million for projects throughout brooklyn i fought for this money will help provide food for the hungry fund community healthcare initiatives support organizations working to uplift our neighborhoods through youth development and empowerment deepen our cultural understanding and more to learn more about these projects and to keep up to date with my work both in new york and washington be sure to subscribe to my communications,Office of Congressman Hakeem Jeffries (NY-08),facebook
1,over the last 25 years america has virtually eliminated plane crashes before that we were on track to see one passenger jet crash per week so what changed our new video explains,VMTS EDUCATION INC,facebook
2,𝗬𝗢𝗨 𝗥𝗘 𝗜𝗡𝗩𝗜𝗧𝗘𝗗 𝘁𝗼 𝗮 𝗳𝗿𝗲𝗲 𝗹𝗶𝘃𝗲𝘀𝘁𝗿𝗲𝗮𝗺 𝗲𝘃𝗲𝗻𝘁 𝗼𝗻 𝘁𝗵𝗲 𝗳𝘂𝘁𝘂𝗿𝗲 𝗼𝗳 𝘁𝗵𝗲 𝗰𝗹𝗶𝗺𝗮𝘁𝗲 𝗺𝗼𝘃𝗲𝗺𝗲𝗻𝘁 𝗠𝗮𝘆 𝟯 𝟭 𝟮 𝟯𝟬 𝗽𝗺 𝗘𝗦𝗧 new york governor kathy hochul joins north carolina governor roy cooper and u s department of energy secretary jennifer granholm to talk about what the federal government and state legislatures are doing to combat climate change right now,Natural Resources Defense Council,facebook
3,what s next in the climate movement find out at a livestream event with new york governor kathy hochul north carolina governor roy cooper and energy secretary jennifer granholm on may 3 1 2 30 pm est welcome remarks from nrdc president manish bapna,Natural Resources Defense Council,facebook
4,we re at a critical turning point in the future of our climate join new york governor kathy hochul at a special livestream event to find out how states and the federal government are taking action on climate tuesday may 3 1 2 30 pm est,Natural Resources Defense Council,facebook
5,we re at a critical turning point in the future of our climate join nrdc president manish bapna at a special livestream event to find out how states and the federal government are taking action on climate tuesday may 3 1 2 30 pm est panelists include new york governor kathy hochul north carolina governor roy cooper and u s department of energy secretary jennifer granholm,Natural Resources Defense Council,facebook
6,𝗬𝗢𝗨 𝗥𝗘 𝗜𝗡𝗩𝗜𝗧𝗘𝗗 𝘁𝗼 𝗮 𝗳𝗿𝗲𝗲 𝗹𝗶𝘃𝗲𝘀𝘁𝗿𝗲𝗮𝗺 𝗲𝘃𝗲𝗻𝘁 𝗼𝗻 𝘁𝗵𝗲 𝗳𝘂𝘁𝘂𝗿𝗲 𝗼𝗳 𝘁𝗵𝗲 𝗰𝗹𝗶𝗺𝗮𝘁𝗲 𝗺𝗼𝘃𝗲𝗺𝗲𝗻𝘁 𝗠𝗮𝘆 𝟯 𝟭 𝟮 𝟯𝟬 𝗽𝗺 𝗘𝗦𝗧 nrdc president manish bapna joins new york governor kathy hochul north carolina governor roy cooper and u s department of energy secretary jennifer granholm to talk about what the federal government and state legislatures are doing to combat climate change right now,Natural Resources Defense Council,facebook
7,what s next in the climate movement find out at a livestream event with north carolina governor roy cooper new york governor kathy hochul u s department of energy secretary jennifer granholm and nrdc president manish bapna on may 3 1 2 30 pm est,Natural Resources Defense Council,facebook
8,join us for an important maternaljustice conversation and learn more about the birth equity act you will hear how parents and caregivers building movement on birth maternal reproductive health equity in nyc,MomsRising Together,facebook
9,special thanks to adam weitsman for hosting my team and i at upstate shredding i ve been to the office countless times over the last 20 plus years to see one of my best friends but this was the first time we walked the grounds together at the shredder upstate shredding is the largest privately owned scrap metal company in the north east with over 500 workers and i saw first hand this massive operation in action the complexities and various moving parts all to provide the best service to its customers and it shows those who know adam know he embodies what it means to be a good corporate citizen at every opportunity adam highlighted his talented hard working and very ca

In [29]:
facebook_df.to_csv('results/clean_ny.csv')

In [5]:
# cncat google_df and facebook_df by 'text'
df_text = pd.concat([google_df,facebook_df])
df_text.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6928 entries, 0 to 6810
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   text             6919 non-null   object
 1   advertiser_name  6876 non-null   object
 2   platform         6928 non-null   object
dtypes: object(3)
memory usage: 216.5+ KB


In [6]:
# check the dataset 
if df_text['text'].isnull().sum():
    print (df_text['text'].isnull().sum())
    df_text = df_text[df_text['text'].notnull()]
    df_text.reset_index(drop=True,inplace=True)

9


In [7]:
df_text = clean_text(df_text)
df_text.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   text             299 non-null    object
 1   advertiser_name  266 non-null    object
 2   platform         299 non-null    object
dtypes: object(3)
memory usage: 7.1+ KB


In [8]:
df_text

,text,advertiser_name,platform
0,representative for new york s first congressional district serving you,REP. LEE ZELDIN - U.S. HOUSE OF REPRESENTATIVES,google
1,new yorkers deserve a governor who works as hard as they do it s why gov kathy hochul has signed over 400 bills in her first six months,Friends for Kathy Hochul,google
2,help khaled to fight the three enemies of humanity poverty ignorance and disease simply put your vote count to get free education in universities for american students,KHALED SALEM,google
3,simply put your vote count to get free education in universities for american students help khaled to fight the three enemies of humanity poverty ignorance and disease,KHALED SALEM,google
4,chip in today and help robert fight for our community in washington robert needs your help to stop corporate pacs from buying this seat,ZIMMERMAN FOR CONGRESS,google
...,...,...,...
294,crime is running rampant in our streets and our communities are suffering in congress you can count on me to spearhead the efforts to fully support law enforcement and help tackle this issue head on,Devolder Santos for Congress,facebook
295,i am grateful to have qualified for the nrcc young guns on the radar list i look forward to helping take back the house,Devolder Santos for Congress,facebook
296,westchester county news county executive latimer proves affordable housing is a top priority in westchester install news app trusted by millions to stay informed of latest westchester county local news,NaN,facebook
297,nassau county news gas tax cut in nassau county after minority caucus push install news app trusted by millions to stay informed of latest nassau county local news,NaN,facebook


In [9]:
df_text = get_word_lable(df_text,'lexicon.json')
df_text

,text,advertiser_name,platform,words,m_label
0,representative for new york s first congressional district serving you,REP. LEE ZELDIN - U.S. HOUSE OF REPRESENTATIVES,google,no words,no topic
1,new yorkers deserve a governor who works as hard as they do it s why gov kathy hochul has signed over 400 bills in her first six months,Friends for Kathy Hochul,google,no words,no topic
2,help khaled to fight the three enemies of humanity poverty ignorance and disease simply put your vote count to get free education in universities for american students,KHALED SALEM,google,"education,free education,enemies,american students","education,foreign policy,social programs"
3,simply put your vote count to get free education in universities for american students help khaled to fight the three enemies of humanity poverty ignorance and disease,KHALED SALEM,google,"education,free education,enemies,american students","education,foreign policy,social programs"
4,chip in today and help robert fight for our community in washington robert needs your help to stop corporate pacs from buying this seat,ZIMMERMAN FOR CONGRESS,google,chip in,donation
...,...,...,...,...,...
294,crime is running rampant in our streets and our communities are suffering in congress you can count on me to spearhead the efforts to fully support law enforcement and help tackle this issue head on,Devolder Santos for Congress,facebook,law,governance
295,i am grateful to have qualified for the nrcc young guns on the radar list i look forward to helping take back the house,Devolder Santos for Congress,facebook,guns,safety
296,westchester county news county executive latimer proves affordable housing is a top priority in westchester install news app trusted by millions to stay informed of latest westchester county local news,NaN,facebook,affordable housing,social programs
297,nassau county news gas tax cut in nassau county after minority caucus push install news app trusted by millions to stay informed of latest nassau county local news,NaN,facebook,tax,economic


In [23]:
df_text.to_csv('results/ad_ny.csv')

In [11]:
text_topic = count_topic(df_text)
text_topic

,single_topic,counts
0,no topic,111
1,economic,44
2,governance,43
3,environment,35
4,social and cultural,35
5,health,33
6,donation,26
7,education,26
8,social programs,25
9,safety,24


In [12]:
text_word = count_word(df_text)
text_word

,single_word,counts
0,no words,111
1,health,25
2,law,23
3,climate,17
4,education,17
...,...,...
210,health coverage,1
211,catholic,1
212,affordability,1
213,chipping in,1


In [13]:
df_text['advertiser_name'].value_counts()

Friends for Kathy Hochul             11
Natural Resources Defense Council    10
ZIMMERMAN FOR CONGRESS                9
Devolder Santos for Congress          7
Democracy Initiative                  7
                                     ..
AMERICA-ISRAEL FRIENDSHIP LEAGUE      1
Patriot United Front LLC              1
Arnold Ventures LLC                   1
Hachette Book Group, Inc.             1
Scott McMurrain                       1
Name: advertiser_name, Length: 132, dtype: int64

In [14]:
df_text[df_text['advertiser_name']=='Friends for Kathy Hochul']

,text,advertiser_name,platform,words,m_label
1,new yorkers deserve a governor who works as hard as they do it s why gov kathy hochul has signed over 400 bills in her first six months,Friends for Kathy Hochul,google,no words,no topic
13,gov kathy hochul is working hard and cutting taxes for middle class new york families she s helping working families make ends meet,Friends for Kathy Hochul,google,"taxes,middle class",economic
14,gov kathy hochul is fighting to help new yorkers keep up with rising costs she s cutting taxes for middle class families,Friends for Kathy Hochul,google,"taxes,middle class",economic
30,gov kathy hochul is fighting to help middle class families keep up with rising costs,Friends for Kathy Hochul,google,middle class,economic
41,gov kathy hochul is cutting taxes for middle class new york families,Friends for Kathy Hochul,google,"taxes,middle class",economic
42,gov kathy hochul is working to help new york families keep up with rising costs she s putting middle class families first,Friends for Kathy Hochul,google,middle class,economic
47,gov kathy hochul is working hard and cutting taxes for middle class new york families,Friends for Kathy Hochul,google,"taxes,middle class",economic
60,gov kathy hochul is working hard and cutting taxes for middle class new york families she ll always put working families first,Friends for Kathy Hochul,google,"taxes,middle class",economic
274,first governor brian kemp in georgia signed a discriminatory election law that disproportionately hurts voters of color then governor doug ducey signed an aggressive anti abortion bill into law that severely restricts reproductive rights in arizona and just last month governor ron desantis in florida signed the discriminatory don t say gay bill attacking the lgbtq community we can t let republicans roll back our rights in new york that s why we need to elect governor kathy hochul this november our freedom is at stake please donate 5 or 10 today towards our campaign thank you,Friends for Kathy Hochul,facebook,"law,donate,reproductive rights,abortion,gay","donation,governance,health,social and cultural"
275,throughout the country republican governors have passed extreme legislation to restrict voting rights reproductive rights lgbtq rights and more we can t let new york fall into the wrong hands and follow this path donate 5 to kathy hochul today to help stand up to the gop and protect our fundamental rights,Friends for Kathy Hochul,facebook,"reproductive rights,donate,voting rights","donation,governance,health"


# GOOGLE

In [15]:
google_df = clean_text(google_df)
google_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   text             64 non-null     object
 1   advertiser_name  64 non-null     object
 2   platform         64 non-null     object
dtypes: object(3)
memory usage: 1.6+ KB


In [16]:
google_df = get_word_lable(google_df,'lexicon.json')
google_df.head()

,text,advertiser_name,platform,words,m_label
0,representative for new york s first congressional district serving you,REP. LEE ZELDIN - U.S. HOUSE OF REPRESENTATIVES,google,no words,no topic
1,new yorkers deserve a governor who works as hard as they do it s why gov kathy hochul has signed over 400 bills in her first six months,Friends for Kathy Hochul,google,no words,no topic
2,help khaled to fight the three enemies of humanity poverty ignorance and disease simply put your vote count to get free education in universities for american students,KHALED SALEM,google,"education,free education,enemies,american students","education,foreign policy,social programs"
3,simply put your vote count to get free education in universities for american students help khaled to fight the three enemies of humanity poverty ignorance and disease,KHALED SALEM,google,"education,free education,enemies,american students","education,foreign policy,social programs"
4,chip in today and help robert fight for our community in washington robert needs your help to stop corporate pacs from buying this seat,ZIMMERMAN FOR CONGRESS,google,chip in,donation


In [17]:
google_topic = count_topic(google_df)
google_topic

,single_topic,counts
0,no topic,26
1,donation,14
2,governance,9
3,economic,8
4,safety,5
5,education,2
6,environment,2
7,foreign policy,2
8,social programs,2
9,health,1


In [18]:
google_word = count_word(google_df)
google_word

,single_word,counts
0,no words,26
1,chip in,7
2,middle class,7
3,donate,6
4,taxes,5
5,laws,4
6,filibuster,4
7,gun violence,3
8,gun,3
9,bail,3


# FACEBOOK

In [24]:
facebook_df = clean_text(facebook_df)
facebook_df

,text,advertiser_name,platform,words,m_label
0,over the past two years we have witnessed how covid 19 deepened the financial need of organizations in our community we depend on while highlighting gaps that existed before the public health emergency in march president biden signed a government funding law that included 21 9 million for projects throughout brooklyn i fought for this money will help provide food for the hungry fund community healthcare initiatives support organizations working to uplift our neighborhoods through youth development and empowerment deepen our cultural understanding and more to learn more about these projects and to keep up to date with my work both in new york and washington be sure to subscribe to my communications,Office of Congressman Hakeem Jeffries (NY-08),facebook,"law,covid 19,healthcare,covid,health","covid,governance,health"
1,over the last 25 years america has virtually eliminated plane crashes before that we were on track to see one passenger jet crash per week so what changed our new video explains,VMTS EDUCATION INC,facebook,no words,no topic
2,𝗬𝗢𝗨 𝗥𝗘 𝗜𝗡𝗩𝗜𝗧𝗘𝗗 𝘁𝗼 𝗮 𝗳𝗿𝗲𝗲 𝗹𝗶𝘃𝗲𝘀𝘁𝗿𝗲𝗮𝗺 𝗲𝘃𝗲𝗻𝘁 𝗼𝗻 𝘁𝗵𝗲 𝗳𝘂𝘁𝘂𝗿𝗲 𝗼𝗳 𝘁𝗵𝗲 𝗰𝗹𝗶𝗺𝗮𝘁𝗲 𝗺𝗼𝘃𝗲𝗺𝗲𝗻𝘁 𝗠𝗮𝘆 𝟯 𝟭 𝟮 𝟯𝟬 𝗽𝗺 𝗘𝗦𝗧 new york governor kathy hochul joins north carolina governor roy cooper and u s department of energy secretary jennifer granholm to talk about what the federal government and state legislatures are doing to combat climate change right now,Natural Resources Defense Council,facebook,"climate,climate change",environment
3,what s next in the climate movement find out at a livestream event with new york governor kathy hochul north carolina governor roy cooper and energy secretary jennifer granholm on may 3 1 2 30 pm est welcome remarks from nrdc president manish bapna,Natural Resources Defense Council,facebook,climate,environment
4,we re at a critical turning point in the future of our climate join new york governor kathy hochul at a special livestream event to find out how states and the federal government are taking action on climate tuesday may 3 1 2 30 pm est,Natural Resources Defense Council,facebook,climate,environment
...,...,...,...,...,...
231,crime is running rampant in our streets and our communities are suffering in congress you can count on me to spearhead the efforts to fully support law enforcement and help tackle this issue head on,Devolder Santos for Congress,facebook,law,governance
232,i am grateful to have qualified for the nrcc young guns on the radar list i look forward to helping take back the house,Devolder Santos for Congress,facebook,guns,safety
233,westchester county news county executive latimer proves affordable housing is a top priority in westchester install news app trusted by millions to stay informed of latest westchester county local news,NaN,facebook,affordable housing,social programs
234,nassau county news gas tax cut in nassau county after minority caucus push install news app trusted by millions to stay informed of latest nassau county local news,NaN,facebook,tax,economic


In [20]:
facebook_df = get_word_lable(facebook_df,'lexicon.json')
facebook_df.head()

,text,advertiser_name,platform,words,m_label
0,over the past two years we have witnessed how covid 19 deepened the financial need of organizations in our community we depend on while highlighting gaps that existed before the public health emergency in march president biden signed a government funding law that included 21 9 million for projects throughout brooklyn i fought for this money will help provide food for the hungry fund community healthcare initiatives support organizations working to uplift our neighborhoods through youth development and empowerment deepen our cultural understanding and more to learn more about these projects and to keep up to date with my work both in new york and washington be sure to subscribe to my communications,Office of Congressman Hakeem Jeffries (NY-08),facebook,"law,covid 19,healthcare,covid,health","covid,governance,health"
1,over the last 25 years america has virtually eliminated plane crashes before that we were on track to see one passenger jet crash per week so what changed our new video explains,VMTS EDUCATION INC,facebook,no words,no topic
2,𝗬𝗢𝗨 𝗥𝗘 𝗜𝗡𝗩𝗜𝗧𝗘𝗗 𝘁𝗼 𝗮 𝗳𝗿𝗲𝗲 𝗹𝗶𝘃𝗲𝘀𝘁𝗿𝗲𝗮𝗺 𝗲𝘃𝗲𝗻𝘁 𝗼𝗻 𝘁𝗵𝗲 𝗳𝘂𝘁𝘂𝗿𝗲 𝗼𝗳 𝘁𝗵𝗲 𝗰𝗹𝗶𝗺𝗮𝘁𝗲 𝗺𝗼𝘃𝗲𝗺𝗲𝗻𝘁 𝗠𝗮𝘆 𝟯 𝟭 𝟮 𝟯𝟬 𝗽𝗺 𝗘𝗦𝗧 new york governor kathy hochul joins north carolina governor roy cooper and u s department of energy secretary jennifer granholm to talk about what the federal government and state legislatures are doing to combat climate change right now,Natural Resources Defense Council,facebook,"climate,climate change",environment
3,what s next in the climate movement find out at a livestream event with new york governor kathy hochul north carolina governor roy cooper and energy secretary jennifer granholm on may 3 1 2 30 pm est welcome remarks from nrdc president manish bapna,Natural Resources Defense Council,facebook,climate,environment
4,we re at a critical turning point in the future of our climate join new york governor kathy hochul at a special livestream event to find out how states and the federal government are taking action on climate tuesday may 3 1 2 30 pm est,Natural Resources Defense Council,facebook,climate,environment


In [21]:
facebook_topic = count_topic(facebook_df)
facebook_topic

,single_topic,counts
0,no topic,86
1,economic,36
2,social and cultural,35
3,governance,34
4,environment,33
5,health,32
6,education,24
7,social programs,23
8,safety,19
9,foreign policy,18


In [22]:
facebook_word = count_word(facebook_df)
facebook_word

,single_word,counts
0,no words,86
1,health,24
2,law,22
3,climate,16
4,education,15
...,...,...
200,health coverage,1
201,carry guns,1
202,gun violence,1
203,catholic,1
